Helpful Links
 - Download and Extract Files: https://pythonguides.com/download-zip-file-from-url-using-python/
 - Leveraging previous code: https://n4maticist.com/articles/2020-01-mcr-data-preparation/
 - Cost Report Field Names: https://gitfreak.com/ohana-project

Import Standard Modules and Functions

In [1]:
import os
from pathlib import Path
from matplotlib.ticker import PercentFormatter
import openpyxl
from datetime import date

# Download ZIP Files
import requests, zipfile
from io import BytesIO

path = os.getcwd()
os.chdir('/Users/CanonPattillo/OneDrive - Porter/Desktop/Stats and DS/Canon Functions')

from canon_functions import *
import canon_functions as cf

os.chdir(path)
print(path)

pd.set_option('display.max_columns', None)

C:\Users\CanonPattillo\OneDrive - Porter\Desktop\Analysis Central\Sales Support and PlanSystem Intelligence\Version Two\Medicare Cost Reports


Download and Extract Medicare Cost Report Raw Data

In [2]:
FYyear = input("Which Fiscal Year (YYYY) would you like to process? >>>  ")

Which Fiscal Year (YYYY) would you like to process? >>>  2020


In [3]:
raw_path = 'C:/Users/CanonPattillo/OneDrive - Porter/Desktop/Analysis Central/Sales Support and PlanSystem Intelligence/Version Two/Medicare Cost Reports/Raw Cost Report Data'

In [ ]:
print('Downloading started')

#Defining the zip file URL
url = f'https://downloads.cms.gov/FILES/HCRIS/HOSP10FY{FYyear}.ZIP'

# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)
print(f'{filename} Download Completed')

# extracting the zip file contents
zipfile.ZipFile(BytesIO(req.content))\
.extractall(raw_path)
print(f'{filename} Extraction Completed')

Import Medicare Cost Report Raw Data

In [4]:
os.chdir(raw_path)
os.listdir()

['HOSP10_2020_ALPHA.CSV', 'HOSP10_2020_NMRC.CSV', 'HOSP10_2020_RPT.CSV']

In [5]:
# Define Columns Names
colRPT = ['RPT_REC_NUM','PRVDR_CTRL_TYPE_CD','PRVDR_NUM','NPI','RPT_STUS_CD', \
            'FY_BGN_DT','FY_END_DT','PROC_DT','INITL_RPT_SW','LAST_RPT_SW', \
            'TRNSMTL_NUM','FI_NUM','ADR_VNDR_CD','FI_CREAT_DT','UTIL_CD','NPR_DT', \
            'SPEC_IND','FI_RCPT_DT']

colNMRC = ['RPT_REC_NUM','WKSHT_CD','LINE_NUM','CLMN_NUM','ITM_VAL']

colALPHA = ['RPT_REC_NUM','WKSHT_CD','LINE_NUM','CLMN_NUM','ITM_VAL']

In [6]:
# Its important to import field as strings to maintain leading zeros.

In [84]:
dfALPHA = pd.read_csv(raw_path + f'/HOSP10_{FYyear}_ALPHA.CSV', \
    header=None, names=colALPHA, dtype='str')

dfNMRC = pd.read_csv(raw_path + f'/HOSP10_{FYyear}_NMRC.CSV', \
    header=None, names=colNMRC, dtype='str')

dfRPT = pd.read_csv(raw_path + f'/HOSP10_{FYyear}_RPT.CSV', \
    header=None, names=colRPT, dtype='str')

Control Totals
 - Check against control totals provided by CMS in HOSP10_RECORD_COUNTS.csv

In [85]:
display(len(dfALPHA), len(dfNMRC), len(dfRPT))

3190230

17047800

5271

Import Field Headers

In [86]:
dfFIELDS = pd.read_csv(raw_path[:-20] + 'Outputs/FIELD_HEADERS.csv',
                       dtype='str')

Import RPT Headers

In [87]:
rptHEADERS = pd.read_csv(raw_path[:-20] + 'Cost Report Documentation/HCRIS_DataDictionary.csv')

Joins

Add Readable Field Names

In [88]:
#%% Merge Field Information with Numeric Data
dfNMRC = dfNMRC.merge(dfFIELDS, left_on=['WKSHT_CD','LINE_NUM','CLMN_NUM'], \
    right_on=['WKSHT_CD','LINE_NUM','CLMN_NUM'])

#%% Merge Field Information with Alpha Data
dfALPHA = dfALPHA.merge(dfFIELDS, left_on=['WKSHT_CD','LINE_NUM','CLMN_NUM'], \
    right_on=['WKSHT_CD','LINE_NUM','CLMN_NUM'])

Join Alpha and Nmrc to Rpt

In [89]:
dfNMRC = dfNMRC.merge(dfRPT, left_on='RPT_REC_NUM', right_on='RPT_REC_NUM')
dfALPHA = dfALPHA.merge(dfRPT, left_on='RPT_REC_NUM', right_on='RPT_REC_NUM')

Create the Full MCR

In [90]:
dfFULL = pd.concat([dfALPHA,dfNMRC]).reset_index(drop=True)

Create the Final MCR

In [91]:
dfFINAL = dfFULL.rename(columns = {'ITM_VAL':'ITM_VAL_NUM'})

In [92]:
plainHEADERS = dict(zip(rptHEADERS['Column Code'],rptHEADERS['Title']))

In [93]:
dfFINAL = dfFINAL.rename(columns = plainHEADERS)

Facilities with > 1x MCR for the FY

In [94]:
# ***** The below two lines demostrate an example where a hospital reported >1x for a FY. We are making the 
# assumption that we should keep the most recent cost report. 

# dfFINAL[(dfFINAL['Provider Number'] == '454148') & (dfFINAL['10 FIELD NAME'] == 'S2_1_C1_1')]
# dfFINAL[dfFINAL['Provider Number'] == '454148']['Report Record Number'].nunique() # 2

# for rrn in dfFINAL[dfFINAL['Provider Number'] == '454148']['Report Record Number'].unique().tolist():
#     print(f"{rrn} unique processing date(s):")
#     print(dfFINAL[(dfFINAL['Provider Number'] == '454148') & (dfFINAL['Report Record Number'] == rrn)]['Process Date'].unique())

# ***** These are the specific facilities with > 1 MCR for the FY

# moreThanOneReport = dfFINAL.groupby(['Provider Number'], as_index=False)['Report Record Number'].nunique()\
# .sort_values('Report Record Number', ascending = False)

# moreThanOneReport = moreThanOneReport[moreThanOneReport['Report Record Number'] >1]

# greThanOne = moreThanOneReport['Provider Number'].unique().tolist()

Export

In [95]:
dfFINAL.to_csv(raw_path[:-20] + 'Outputs/' + f'Unprocessed_{FYyear}_Medicare_Cost_Report.csv', index = False)